## instanceof
[Java 16](https://openjdk.org/jeps/394) introduced improvement to `instanceof` operator where instead of writing:

In [ ]:
if(obj instanceof String) {
    String str = (String) obj;
    // ...
}

we can shorten it to:

In [ ]:
if(obj instanceof String str) {
    // ...
}

// str not in scope here

We can further extend the test:

In [ ]:
// We can't replace && with || here
if(obj instanceof String str && str.length() > 0) {
    // ...
}

## Switch Statement
Switch case statement in Java has had multiple limits. `switch` can only accept the following types:
- `byte` and `Byte`
- `short` and `Short`
- `char` and `Character`
- `int` and `Integer`
- Enums
- `String` (Java 7)

Which means the following statement was not permitted:

In [ ]:
long l = 123456l;
switch(l) {  // --> Error
    case 0l:
        // ...
}

There can also be scenarios involving NPE:

In [ ]:
String s = null;
switch(s) {  // Results in NPE
    case "Java":
        System.out.println("Programming language of choice");
        break;
    default:
        System.out.println("Meh");
        break;
}

/*
In Java 21, we can remedy above by:
String s = null;
switch(s) {
    case "Java":
        System.out.println("Programming language of choice");
        break;
    case null:  // Adding a null case
        System.out.println("Input is null");
        break;
    default:
        System.out.println("Meh");
        break;
}
*/

Case labels are fall-through:

In [ ]:
switch(input) {
    case 0:
    case 1:
        answer = 1;
        break;
    default:
        answer = 0;
        break;
}

Case labels must have constants:

In [ ]:
case getValue(): // Error

Switch with case labels does not enforce exhaustiveness check:

In [ ]:
switch (p) {
    case EARTH:
        System.out.println("On earth");
}

[Java 14](https://openjdk.org/jeps/361) introduced new form of case label called *arrow label*:

In [ ]:
switch(day) {
    case 1,2,3,4,5 -> System.out.println("Work to do"); // No fall through
    case 1,6 -> System.out.println("Rest");
}

[Java 21](https://openjdk.org/jeps/441) introduced *pattern matching* which means we can put any object in the `switch`:

In [ ]:
public int toInt(Object o) {
    int answer;
    switch (o) {
        case Integer i -> answer = i;
        case Long l -> answer = l.intValue();
        case Float f -> answer = f.intValue();
        case Double d -> answer = d.intValue();
        case String s -> answer = Integer.parseInt(s);
        case null -> answer = 0; // default does not match null
        default -> throw new IllegalStateException("Unexpected value: " + o);
    }
    return answer;
}

**Exhaustiveness Check:** Java performs exhaustiveness check if we use patterns in arrow label (like in the above example) or if we use *switch expressions* (discussed later). This means that the above code must have `default` label. Why does the below code require `default`?

In [ ]:
public boolean isBlank(String input) {
    boolean result = false;
    switch (input) {
        case "", " ", "\t", "\n", "\r" -> result = true;
        case null -> result = true;
        default -> {} // default does not match null
    }
    return result;
}

This is because we added `case null` which is part of the pattern-matching `switch` syntax.

**Case Refinement:** we can add `when` clauses in switch blocks to specify guards to pattern case labels:

In [ ]:
switch(str) {
    case null -> {}
    case String s when "Y".equals(s) -> System.out.println("Yes");
    case String s when "N".equals(s) -> System.out.println("No");
    default -> System.out.println("Unknown input");
}

**Dominance Rule:** compared to previous switch statement, now more than one case can match the pattern. So, if we specify a more dominant case first, it results in error:

In [ ]:
switch(input) {
    case Number n -> System.out.println(n);
    case Integer i -> System.out.println(n); // Error
    default -> System.out.println("Not a number");
}

switch(str) {
    String s -> // ...
    String s when s.length() > 1 -> // ... Error
    default -> // ...
}

**Pattern Matching Records:**

In [ ]:
record Pair(int x, int y) {}
record Triple(int x, int y, int z) {}

public void print(Object obj) {
    switch (obj) {
        case Pair(int x, int y) -> System.out.println(x + " " + y);
        case Triple(int x, int y, int z) -> System.out.println(x + " " + y + " " + z);
        default -> throw new IllegalArgumentException("Invalid object type");
    }
}

### Switch Expressions
In Java 14+, `switch` statement has been extended so it can be used as an expression. It follows the same rules as discussed above.

In [ ]:
int sum = switch (obj) { // Must be exhaustive
    case Pair(int x, int y) -> x + y;
    case Triple(int x, int y, int z) -> x + y + z;
    default -> 0;
};

The same can also be written using case-labels using `yield` keyword:

In [ ]:
int sum = switch (obj) {
    case Pair(int x, int y): yield x + y;
    case Triple(int x, int y, int z): yield x + y + z;
    default: yield 0;
};

`yield` is also used when block expression is involved to the right of ->:

In [ ]:
int sum = switch (obj) { // Must be exhaustive
    case Pair(int x, int y) -> x + y;
    case Triple(int x, int y, int z) -> x + y + z;
    default -> {
        String s = obj.toString();
        yield s.length();
    };
};

Since switch expressions enforce exhaustiveness, all values of an enum must be checked:

In [ ]:
boolean hasLife = switch (p) {
    case EARTH -> true;
    case MERCURY, VENUS, MARS, JUPITER, SATURN, URANUS, NEPTUNE -> false;
};

## Sealed Classes